In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import shutil
from shutil import copyfile
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from scipy import ceil
import scipy

In [ ]:
source_path = '/content/drive/MyDrive/Data'


list=[]

for i in os.listdir(source_path):
    source_path_item = os.path.join(source_path, i)
    n = 0
    for j in os.listdir(source_path_item):
        if j[-3:] == "jpg" or j[-3:] == "gif":
            im = Image.open(os.path.join(source_path_item, j)).convert("RGB")
            im.save(os.path.join(source_path_item, j[:-3]+'jpeg'))

            os.remove(os.path.join(source_path_item, j))
            n += 1

    print("Convert Successful")
    print("Converted image =",n)
    print(f"There are {len(os.listdir(source_path_item))} images of", i + ".\n")
    list.append(i)

Convert Successful
Converted image = 0
There are 1306 images of Calculus.

Convert Successful
Converted image = 0
There are 2382 images of Data caries.

Convert Successful
Converted image = 0
There are 183 images of Tooth Discoloration.

Convert Successful
Converted image = 0
There are 1251 images of hypodontia.

Convert Successful
Converted image = 0
There are 82 images of normal.

Convert Successful
Converted image = 0
There are 2541 images of Mouth Ulcer.

Convert Successful
Converted image = 0
There are 2349 images of Gingivitis.



In [ ]:
root_dir = './temp'

if os.path.exists(root_dir):
    shutil.rmtree(root_dir)

for i in list:
    os.makedirs(os.path.join(root_dir,"train",i))
    os.makedirs(os.path.join(root_dir,"validation",i))

In [ ]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):

  random_sample=random.sample(os.listdir(SOURCE_DIR),len(os.listdir(SOURCE_DIR)))

  size=int(len(random_sample)*SPLIT_SIZE)

  target=TRAINING_DIR
  i=0

  for item in random_sample:

    item_source = os.path.join(SOURCE_DIR, item)

    if os.path.getsize(item_source) == 0:
      print(f'{item} is zero length, so ignoring.')
    else:
      copyfile(item_source, os.path.join(target, item))
      i += 1

    if i == size:
      target = VALIDATION_DIR

In [ ]:
train_dir = os.path.join(root_dir,"train")
val_dir = os.path.join(root_dir,"validation")

for i in os.listdir(train_dir):
  path = os.path.join(train_dir,i)
  if len(os.listdir(path))>0:
    for file in os.scandir(path):
      os.remove(file.path)

for i in os.listdir(val_dir):
  path = os.path.join(val_dir,i)
  if len(os.listdir(path))>0:
    for file in os.scandir(path):
      os.remove(file.path)

split_size = .8

for i in list:
  split_data(os.path.join(source_path,i), os.path.join(train_dir, i), os.path.join(val_dir, i), split_size)
  print("Split", i, "Successful")
  print(f"There are {len(os.listdir(os.path.join(train_dir,i)))} images of", i + " for training.")
  print(f"There are {len(os.listdir(os.path.join(val_dir,i)))} images of", i + " for validation.\n")

Split Calculus Successful
There are 1044 images of Calculus for training.
There are 262 images of Calculus for validation.

Split Data caries Successful
There are 1905 images of Data caries for training.
There are 477 images of Data caries for validation.

Split Tooth Discoloration Successful
There are 146 images of Tooth Discoloration for training.
There are 37 images of Tooth Discoloration for validation.

Split hypodontia Successful
There are 1000 images of hypodontia for training.
There are 251 images of hypodontia for validation.

Split normal Successful
There are 65 images of normal for training.
There are 17 images of normal for validation.

Split Mouth Ulcer Successful
There are 2032 images of Mouth Ulcer for training.
There are 509 images of Mouth Ulcer for validation.

Split Gingivitis Successful
There are 1879 images of Gingivitis for training.
There are 470 images of Gingivitis for validation.



In [ ]:
NUM_CLASSES = len(os.listdir(source_path))

BATCH_SIZE = 60

base_model = tf.keras.applications.VGG16(input_shape=(225,225,3),
                                         include_top=False,
                                         weights='imagenet')

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 225, 225, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 225, 225, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 225, 225, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
last_desired_layer = base_model.get_layer("block5_pool")

last_output = last_desired_layer.output

In [ ]:
for layer in base_model.layers:
 layer.trainable = False
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(256, activation = 'relu')(x)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)
x = tf.keras.layers.Dense(64, activation = 'relu')(x)
x = tf.keras.layers.Dense(32, activation = 'relu')(x)
x = tf.keras.layers.Dense(NUM_CLASSES, activation = 'softmax')(x)

model = tf.keras.models.Model(inputs = base_model.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 225, 225, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 225, 225, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 225, 225, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:

train_datagen = ImageDataGenerator(rescale=1.0/255,
                                     rotation_range=20,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(225,225),
                                                    shuffle=False,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1.0/255,
                                     rotation_range=20,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')
validation_generator = validation_datagen.flow_from_directory(directory=val_dir,
                                                              target_size=(225,225),
                                                              shuffle=False,
                                                              class_mode='categorical')

training_steps_per_epoch = np.ceil(train_generator.samples / BATCH_SIZE)
validation_steps_per_epoch = np.ceil(validation_generator.samples / BATCH_SIZE)

history = model.fit(train_generator,
                    steps_per_epoch = training_steps_per_epoch,
                    validation_data=validation_generator,
                    validation_steps=validation_steps_per_epoch,
                    epochs=25,
                    verbose=1)

print('Training Completed!')

Found 8071 images belonging to 7 classes.
Found 2023 images belonging to 7 classes.
Epoch 1/25
135/135 [==============================] - 93s 602ms/step - loss: 2.1782 - accuracy: 0.2710 - val_loss: 1.6096 - val_accuracy: 0.2684
Epoch 2/25
135/135 [==============================] - 84s 621ms/step - loss: 1.3310 - accuracy: 0.4556 - val_loss: 1.4169 - val_accuracy: 0.2647
Epoch 3/25
135/135 [==============================] - 77s 571ms/step - loss: 1.3076 - accuracy: 0.4083 - val_loss: 1.3002 - val_accuracy: 0.2969
Epoch 4/25
135/135 [==============================] - 83s 615ms/step - loss: 1.1762 - accuracy: 0.4819 - val_loss: 1.0859 - val_accuracy: 0.6241
Epoch 5/25
135/135 [==============================] - 78s 576ms/step - loss: 1.1750 - accuracy: 0.4815 - val_loss: 1.2513 - val_accuracy: 0.4274
Epoch 6/25
135/135 [==============================] - 80s 591ms/step - loss: 1.2489 - accuracy: 0.4794 - val_loss: 1.1306 - val_accuracy: 0.4982
Epoch 7/25
135/135 [==========================